### Sai Srihitha Goverdhana  --------- U58956033

Universal bank has recently trialed a marketing campaign to sell their new CD account product to existing customers. They contacted 5000 of their non-CD account customers with an offer. The data provided in universal.csv is the result of this market test. 

Use the techniques covered in this class to load and clean the data. Then, identify the best predictive model (using only the models covered thus far: Logistic Regression, SVM (with various kernels), and Decision trees). Your target variable is CD Account. Your scoring measure is recall. Use RandomSearchCV combined with GridSearchCV to identify the best parameters for each model tested.

Be sure to document your thought process using markdown. Think of this as a report that your manager will read. This assignment requires you to decide how to process the provided data best (i.e., encoding). Be sure to provide your arguments/observations in markdown as you progress through data preparation, fitting, and performance evaluation.

## 1. Setup

Import all the Required Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

np.random.seed(1)

## 2. Load the Data

In [2]:
import os
print(os.getcwd())

C:\Users\srihi\DSP\Week 4


In [3]:
df = pd.read_csv('C:/Users/srihi/DSP/data/UniversalBank.csv')
df.head(3)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0


In [4]:
df.dtypes

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object

In [5]:
df['CD Account'].unique()

array([0, 1], dtype=int64)

### Explore Variables

In [6]:
df.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [8]:
df.columns = df.columns.str.replace(' ', '_')

In [9]:
df.columns = df.columns.str.lower()
df.head(5)

,id,age,experience,income,zip_code,family,ccavg,education,mortgage,personal_loan,securities_account,cd_account,online,creditcard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [10]:
df[['cd_account']].value_counts()

cd_account
0             4698
1              302
dtype: int64

In [11]:
import summarytools
from summarytools import dfSummary
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,id[int64],Mean (sd) : 1.9 (1.7)min < med < max:0.0 < 1.5 < 10.0IQR (CV) : 1.8 (1.1),108 distinct values,,0(0.0%)
2,age[int64],nan,nan,nan,0(0.0%)
3,experience[int64],nan,nan,nan,0(0.0%)
4,income[int64],nan,nan,nan,0(0.0%)
5,zip_code[int64],nan,nan,nan,0(0.0%)
6,family[int64],nan,nan,nan,0(0.0%)
7,ccavg[float64],nan,nan,nan,0(0.0%)
8,education[int64],nan,nan,nan,0(0.0%)
9,mortgage[int64],nan,nan,nan,0(0.0%)
10,personal_loan[int64],nan,nan,nan,0(0.0%)


In [12]:
# Check the missing values by summing the total na's for each variable
df.isna().sum()

id                    0
age                   0
experience            0
income                0
zip_code              0
family                0
ccavg                 0
education             0
mortgage              0
personal_loan         0
securities_account    0
cd_account            0
online                0
creditcard            0
dtype: int64

In [13]:
df.isnull().sum().sum()

0

> In our data there are no null values to be handled

## Split and Standardize Data

In [14]:
# First split the data into X,y 
X = df.drop('cd_account', axis=1)
y = df['cd_account']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)

# Initialise the scaler
scaler = StandardScaler()

# Fit the scaler to the training set
scaler.fit(X_train)

# Transform the training and testing sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


### Model the Data

In [15]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### Fit a Logistic Regression Model

In [16]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [17]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [18]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978,1.0,0.60241,0.75188


### Logistic Regression - RandomSearchCV 

In [20]:
#prepare the hyperparameter list to tune
hyperparameter_list = {'C':[0.001, 0.01, 0.1, 1, 10, 100]
                     }

score_measure = "recall"

#create a random search cv object
random_search_cv_log = RandomizedSearchCV(estimator=log_reg_model,
                                     param_distributions=hyperparameter_list,
                                     cv=5,
                                          scoring=score_measure,
                                     n_iter=10,
                                     n_jobs=-1
                                    )

_ = random_search_cv_log.fit(X_train, y_train)

print(f"The best {score_measure} score is {random_search_cv_log.best_score_}")
print(f"... with parameters: {random_search_cv_log.best_params_}")

bestRecallTree = random_search_cv_log.best_estimator_


The best recall score is 0.6662790697674419
... with parameters: {'C': 0.1}


> After RandomSearchCV on training data - The best Recall score is 66.62%

In [21]:
c_matrix = confusion_matrix(y_test, random_search_cv_log.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
# print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

performance = pd.concat([performance, pd.DataFrame({'model':"Log Reg RandSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [22]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978,1.0,0.60241,0.75188
0,Log Reg RandSearch,0.978,1.0,0.60241,0.75188


### Logistic Regression - GridSearchCV 

In [23]:
score_measure = "recall"

#Define the parameter grid
param_grid = {
    'C':[0.1,1,10]
}

grid_search = GridSearchCV(estimator=log_reg_model, param_grid=param_grid, cv = 5, scoring=score_measure)


_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

The best recall score is 0.6662790697674419
... with parameters: {'C': 0.1}


> After GridSearchCV on training data - The best Recall score is 66.62% which is same as recall value of RandomSearchCV for logistic Regression

In [24]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Log Reg GridSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978,1.0,0.60241,0.75188
0,Log Reg RandSearch,0.978,1.0,0.60241,0.75188
0,Log Reg GridSearch,0.978,1.0,0.60241,0.75188


### 3.1 Fit a SVM classification model using linear kernal

In [25]:
svm_lin_model = SVC(kernel="linear")
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [26]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"linear svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978,1.0,0.60241,0.75188
0,Log Reg RandSearch,0.978,1.0,0.60241,0.75188
0,Log Reg GridSearch,0.978,1.0,0.60241,0.75188
0,linear svm,0.978,1.0,0.60241,0.75188


### SVM Linear - RandomSearchCV

In [27]:
score_measure = "recall"
# Create parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}


# Create the random search model
rand_search = RandomizedSearchCV(svm_lin_model, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best recall score is 0.6800211416490486
... with parameters: {'max_iter': 1500, 'C': 100}


> There is slight increase in best Recall score for model SVM Linear Kernel which is 68%. 

In [28]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM linear RandSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887


### SVM Linear - GridSearchCV

In [29]:
score_measure = "recall"
kfolds = 5

#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear']}  
#instantiate the GridSearchCV object
grid = GridSearchCV(svm_lin_model, param_grid, cv=5,scoring=score_measure)


_ = grid.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_


The best recall score is 0.6662790697674419
... with parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}


> Here the best recall score for SVM Linear GridSerachCV model is 66.62

In [30]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM linear GridSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880


### 3.2 Fit a SVM classification model using rbf kernal

In [31]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale')
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [32]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rbf svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [33]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333


### SVM rbf - RandomSearchCV

In [34]:
from sklearn.model_selection import RandomizedSearchCV

score_measure = "recall"
# Create parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

# Create the random search model
rand_search_rbf = RandomizedSearchCV(svm_rbf_model, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = rand_search_rbf.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_rbf.best_score_}")
print(f"... with parameters: {rand_search_rbf.best_params_}")

bestRecallTree = rand_search_rbf.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best recall score is 0.6849894291754757
... with parameters: {'max_iter': 1500, 'C': 100}


> There is slight increase in best Recall score for model SVM rbf Kernel which is 68.49%. 

In [35]:
c_matrix = confusion_matrix(y_test, rand_search_rbf.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf RandSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943


### SVM rbf - GridSearchCV

In [36]:
score_measure = "recall"

#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['rbf']}  
#instantiate the GridSearchCV object
grid = GridSearchCV(svm_rbf_model, param_grid, cv=5, scoring=score_measure)


_ = grid.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_


The best recall score is 0.666384778012685
... with parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


>  Here the best recall score for SVM Linear GridSerachCV model is 66.63

In [37]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf GridSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444


### 3.3 Fit a SVM classification model using polynomial kernal

In [38]:
 = SVC(kernel="poly", degree=3, coef0=1, C=10, probability = True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [39]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [40]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444
0,poly svm,0.962667,0.684932,0.602410,0.641026


### SVM Poly - RandomSearchCV

In [41]:
score_measure = "recall"
# Create parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}


# Create the random search model
rand_search_poly = RandomizedSearchCV(svm_poly_model, param_grid, cv=5, n_iter=50, scoring='recall', n_jobs=-1, verbose=1)


_ = rand_search_poly.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_poly.best_score_}")
print(f"... with parameters: {rand_search_poly.best_params_}")

bestRecallTree = rand_search_poly.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best recall score is 0.739429175475687
... with parameters: {'max_iter': 1000, 'C': 100}


> 73.94% is the best recall score for SVM Poly RandomSearchCV

In [42]:
c_matrix = confusion_matrix(y_test, rand_search_poly.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM Poly RandSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444
0,poly svm,0.962667,0.684932,0.602410,0.641026


### SVM Poly - GridSearchCV

In [43]:
score_measure = "recall"
#create a SVM classifier
clf = SVC(kernel='poly')
#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['poly']}  
#instantiate the GridSearchCV object
grid = GridSearchCV(clf, param_grid, cv=5,scoring=score_measure)


_ = grid.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_


The best recall score is 0.6849894291754757
... with parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}


> Best Recall Score for SVM Poly for GridSearchCV is 68.49

In [44]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM poly GridSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444
0,poly svm,0.962667,0.684932,0.602410,0.641026


## 3.1 DTree classification model using defaults 

In [45]:
dtree = DecisionTreeClassifier()
_ = dtree.fit(X_train, np.ravel(y_train))

In [46]:
model_preds = dtree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444
0,poly svm,0.962667,0.684932,0.602410,0.641026


### DTree - RandomSearchCV

In [47]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.6753699788583509
... with parameters: {'min_samples_split': 13, 'min_samples_leaf': 2, 'min_impurity_decrease': 0.0006000000000000001, 'max_leaf_nodes': 55, 'max_depth': 23, 'criterion': 'gini'}


In [48]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DTree RandSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444
0,poly svm,0.962667,0.684932,0.602410,0.641026


### DTree - GridSearchCV

In [49]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(30,36),  
    'min_samples_leaf': np.arange(6,12),
    'min_impurity_decrease': np.arange(0.0048, 0.0054, 0.0001),
    'max_leaf_nodes': np.arange(162,168), 
    'max_depth': np.arange(15,21), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 9072 candidates, totalling 45360 fits
The best recall score is 0.6480972515856237
... with parameters: {'criterion': 'entropy', 'max_depth': 15, 'max_leaf_nodes': 162, 'min_impurity_decrease': 0.0048, 'min_samples_leaf': 6, 'min_samples_split': 30}


In [53]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DTree GridSearch", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear RandSearch,0.896667,0.293103,0.614458,0.396887
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.972000,0.836066,0.614458,0.708333
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444


#### For easy comparision , we can sort the recall value as below

In [52]:
performance.sort_values(by=['Recall'])

,model,Accuracy,Precision,Recall,F1
0,SVM poly GridSearch,0.957333,0.623377,0.578313,0.600000
0,default logistic,0.978000,1.000000,0.602410,0.751880
0,Log Reg RandSearch,0.978000,1.000000,0.602410,0.751880
0,Log Reg GridSearch,0.978000,1.000000,0.602410,0.751880
0,linear svm,0.978000,1.000000,0.602410,0.751880
0,SVM linear GridSearch,0.978000,1.000000,0.602410,0.751880
0,SVM rbf RandSearch,0.962000,0.675676,0.602410,0.636943
0,SVM rbf GridSearch,0.970667,0.819672,0.602410,0.694444
0,poly svm,0.962667,0.684932,0.602410,0.641026
0,DTree RandSearch,0.974667,0.909091,0.602410,0.724638


# Analysis Report:

### According to the above Performance Dataframe , considering Recall as Performance metric - After training and testing the data on several different models and different combinations such as Logistic regression , SVM with Different Kernels and Decision Tree

### Deafualt Decision tree and SVM with kernal Poly using RandomSearchCV recored highest recall wich is "62.65%". 

### Second Highest Recall value is 61.44 for models such as SVM Linear Kernal with RandomSearchCV and Svm with rbf Kernal

## So the best model according the model performance is with Default Decision Tree and SVM with Poly Kernal RandomizedCV models